# Read, Parse, Process E-Commerce data on PySpark
eCommerce dataset: https://www.kaggle.com/mkechinov/ecommerce-behavior-data-from-multi-category

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

## Data Download from Kaggle

In [3]:
# ! pip install kaggle --upgrade

     |████████████████████████████████| 58 kB 2.0 MB/s eta 0:00:011
     |████████████████████████████████| 118 kB 11.3 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 12.1 MB/s eta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=f6df588ba0dce2fdaa560c7260b6f24d4c3d19dddd40a82c5f8e59d4fb7e44e1
  Stored in directory: /home/jovyan/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
  Created wheel for python-slugify: filename=python_slugify-4.0.0-py2.py3-none-any.whl size=5486 sha256=ccd5fc462f4c2e3aff0d34f878d667cdf53acd018d64939570e8ce3c1ede5e4e
  Stored in directory: /home/jovyan/.cache/pip/wheels/7c/26/30/5f3d95da00fe94d0c4a5ec5b4ffd2e1ae18545f5fa61752e52
Successfully built kaggle python-slugify
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.9
    Uninstalling urllib3-1.25.9:
      Successfully uninstalled urllib3-1.25.9


In [17]:
# NOTE: first to get kaggle api tiken from account page in Kaggle. Place it at ~/.kaggle/kaggle.json
# mkdir ~/.kaggle/
# nano ~/.kaggle/kaggle.json
# chmod 600 ~/.kaggle/kaggle.json
# kaggle datasets download mkechinov/ecommerce-behavior-data-from-multi-category-store
# unzip ecommerce-behavior-data-from-multi-category-store.zip

## Setup Spark

In [2]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS
from pyspark.sql import Window
from pyspark.ml.linalg import Vectors, Vector, DenseVector, SparseVector, VectorUDT
from pyspark.ml.stat import Summarizer

In [3]:
from pyspark.sql import SparkSession

# Setup for 400GB Mem machine
config = pyspark.SparkConf().setAll([('spark.executor.memory', '350g'),
                                     ('spark.executor.instances', '1'),
                                     ('spark.cores.max', '64'),
                                     #('spark.cores.max', '3'),
                                     ('spark.executor.cores', '64'),
                                     ('spark.driver.memory','30g')
                                    ])
spark = SparkSession.builder.config(conf=config).appName("JobName").getOrCreate()

# Setup for 200GB Mem machine
# config = pyspark.SparkConf().setAll([('spark.executor.cores', '4'),
#                                      ('spark.executor.instances', '2'),
#                                      ('spark.executor.memory', '12g'), 
#                                      ('spark.executor.memoryOverhead', '2g'), 
#                                      #('spark.driver.cores', '4'),
#                                      #('spark.driver.memory', '12g'),
#                                      ('spark.dynamicAllocation.enabled', 'false'),
#                                      ('spark.io.compression.codec', 'snappy')
#                                     ])

# spark = SparkSession.builder.config(conf=config).appName("Recsys-Transformer-Research").getOrCreate()

## Read Data

#### Load 4 months dataset

In [4]:
DATA_PATH = '/home/jovyan/ecommerce_raw'

In [5]:
data_csv_01 = os.path.join(DATA_PATH, "2019-Oct.csv")
data_csv_02 = os.path.join(DATA_PATH, "2019-Nov.csv")
# data_csv_03 = "/workspace/ecommerce-behavior-data-from-multi-category-store/2019-Dec.csv"
# data_csv_04 = "/workspace/ecommerce-behavior-data-from-multi-category-store/2020-Jan.csv"

data_list = [data_csv_01, data_csv_02]

#### Read through Spark from CSV

In [6]:
_df_sb = spark.read.csv(data_list, inferSchema = True, header = True) #.limit(1000)
_df_sb.printSchema()

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



#### Convert timestamp as the type of datetime

In [7]:
_df_sb_a = _df_sb.withColumn('event_time_dt', F.to_timestamp('event_time','yyyy-MM-dd HH:mm:ss')) \
                 .withColumn('event_time_ts', F.unix_timestamp('event_time_dt')) \
                 .withColumn('et_hour', F.hour('event_time_dt'))\
                 .withColumn('et_dayofweek', F.dayofweek('event_time_dt'))\
                 .withColumn('et_dayofmonth', F.dayofmonth('event_time_dt'))\
                 .withColumn('et_month', F.month('event_time_dt'))                

In [8]:
def cyclical_feature(value, func, max_value):
    if func == 'sin':
        f = np.sin
    elif func == 'cos':
        f = np.cos
    else:
        raise Exception('Invalid func (expected: sin|cos)')
        
    value_scaled = (value + 1e-8) / max_value
    return float(f(2.*np.pi*value_scaled))

@udf(returnType=FloatType())
def cyclical_feature_udf(value, func, max_value):
    return cyclical_feature(value, func, max_value)

In [9]:
# Generating cyclical features to model continuity on temporal features
_df_sb_a = _df_sb_a.withColumn('et_hour_sin', cyclical_feature_udf(F.col('et_hour'), F.lit('sin'), F.lit(24))) \
                   .withColumn('et_hour_cos', cyclical_feature_udf(F.col('et_hour'), F.lit('cos'), F.lit(24))) \
                   .withColumn('et_dayofweek_sin', cyclical_feature_udf(F.col('et_dayofweek'), F.lit('sin'), F.lit(7))) \
                   .withColumn('et_dayofweek_cos', cyclical_feature_udf(F.col('et_dayofweek'), F.lit('cos'), F.lit(7))) \
                   .withColumn('et_dayofmonth_sin', cyclical_feature_udf(F.col('et_dayofmonth'), F.lit('sin'), F.lit(31))) \
                   .withColumn('et_dayofmonth_cos', cyclical_feature_udf(F.col('et_dayofmonth'), F.lit('cos'), F.lit(31))) \
                   .withColumn('et_month_sin', cyclical_feature_udf(F.col('et_month'), F.lit('sin'), F.lit(12))) \
                   .withColumn('et_month_cos', cyclical_feature_udf(F.col('et_month'), F.lit('cos'), F.lit(12)))

In [150]:
'''
l = []
for i in range(24):
    s = cyclical_feature(i, 'sin', 24)
    c = cyclical_feature(i, 'cos', 24)
    l.append((s,c))
    
df = pd.DataFrame(l, columns=['sin', 'cos']).reset_index()
df.plot.line(x='index', y='sin')
df.plot.line(x='index', y='cos')
'''

In [10]:
# Recency feature
item_first_interaction_df = _df_sb_a.groupBy('product_id').agg(F.min('event_time_ts').alias('prod_first_event_time_ts'))
_df_sb_a = _df_sb_a.join(item_first_interaction_df, on='product_id', how='inner') \
            .withColumn('product_recency_days', (F.col('event_time_ts') - F.col('prod_first_event_time_ts')) / (60*60*24)) \
            .withColumn('product_recency_days_log', F.log1p('product_recency_days'))

In [11]:
#Smoothing price long-tailed distribution
_df_sb_a = _df_sb_a.withColumn('price_log', F.log1p('price'))

In [12]:
#Relative Price to the average price for the category_id
avg_category_id_prices_df = _df_sb_a.groupBy('category_id').agg(F.mean('price').alias('avg_category_id_price'))
_df_sb_a = _df_sb_a.join(avg_category_id_prices_df, on='category_id', how='inner') \
        .withColumn('relative_price_to_avg_category_id', (F.col('price') - F.col('avg_category_id_price')) / F.col('avg_category_id_price'))

In [13]:
#_df_sb_a.groupBy('product_id').agg(F.stddev('price').alias('std')).where(~F.isnan('std')).agg(F.mean('std')).show()

In [14]:
#_df_sb_a.groupBy('category_id').agg(F.stddev('price').alias('std')).where(~F.isnan('std')).agg(F.mean('std')).show()

In [15]:
#_df_sb_a.groupBy('category_code').agg(F.stddev('price').alias('std')).agg(F.mean('std')).show()

#### String -> Discrete Feature

API: https://spark.apache.org/docs/latest/api/python/pyspark.ml.html?pyspark.ml.feature.StringIndexer#pyspark.ml.feature.StringIndexer

How to apply:
https://stackoverflow.com/questions/36942233/apply-stringindexer-to-several-columns-in-a-pyspark-dataframe

In [16]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

#Null values are treated by handleInvalid="keep" with a special bucket
product_idxer = StringIndexer(inputCol="product_id", outputCol="product_idx", handleInvalid="keep", stringOrderType="frequencyDesc")
category_id_idxer = StringIndexer(inputCol="category_id", outputCol="category_sub_idx", handleInvalid="keep", stringOrderType="frequencyDesc")
category_code_idxer = StringIndexer(inputCol="category_code", outputCol="category_code_idx", handleInvalid="keep", stringOrderType="frequencyDesc")
brand_idxer = StringIndexer(inputCol="brand", outputCol="brand_idx", handleInvalid="keep", stringOrderType="frequencyDesc")
#userid_idxer = StringIndexer(inputCol="user_id", outputCol="user_idx", handleInvalid="keep", stringOrderType="frequencyDesc")
event_type_idxer = StringIndexer(inputCol="event_type", outputCol="event_type_idx", handleInvalid="keep", stringOrderType="frequencyDesc")


#NOTE: userid_idxer gets error!

indexers = [product_idxer, category_id_idxer, category_code_idxer, brand_idxer, event_type_idxer]   #userid_idxer
indexing_pipeline = Pipeline(stages = indexers)
         
indexing_model = indexing_pipeline.fit(_df_sb_a)
_df_sb_a_r = indexing_model.transform(_df_sb_a)

In [17]:
#Just to keep the pipeline unchanged after disabling String Indexing for the User
_df_sb_a_r = _df_sb_a_r.withColumn('user_idx', F.col('user_id'))

#### Add +2 to indexed columns

In [18]:
start_idx = 2

_df_sb_a_r = _df_sb_a_r.withColumn('product_idx', (F.col('product_idx')+start_idx)) \
                       .withColumn('category_sub_idx', (F.col('category_sub_idx')+start_idx)) \
                       .withColumn('category_code_idx', (F.col('category_code_idx')+start_idx)) \
                       .withColumn('brand_idx', (F.col('brand_idx')+start_idx)) \
                       .withColumn('event_type_idx', F.col('event_type_idx')+start_idx)          

#.withColumn('user_idx', (F.col('user_idx')+start_idx)) \

### Get statistics

In [19]:
from pyspark.sql.functions import countDistinct
gr = _df_sb_a_r.agg(F.countDistinct("user_session"))
gr.show()

+----------------------------+
|count(DISTINCT user_session)|
+----------------------------+
|                    23016650|
+----------------------------+



In [20]:
gr = _df_sb_a_r.agg(F.countDistinct("user_id"))
gr.show()

+-----------------------+
|count(DISTINCT user_id)|
+-----------------------+
|                5316649|
+-----------------------+



In [21]:
gr = _df_sb_a_r.agg(F.countDistinct("product_id"))
gr.show()

+--------------------------+
|count(DISTINCT product_id)|
+--------------------------+
|                    206876|
+--------------------------+



In [19]:
gr = _df_sb_a_r.agg(F.countDistinct("category_id"))
gr.show()

+---------------------------+
|count(DISTINCT category_id)|
+---------------------------+
|                        691|
+---------------------------+



#### Normalize Continuous Features

#### Price (log)

In [20]:
%%time
price_log_mean, price_log_std = tuple(_df_sb_a_r.agg(F.mean('price_log'), F.stddev('price_log')).take(1)[0])
print(price_log_mean, price_log_std)

5.039902138802762 1.2186780414302592
CPU times: user 9.74 ms, sys: 7.56 ms, total: 17.3 ms
Wall time: 50.8 s


In [21]:
#Z-norm
_df_sb_a_r = _df_sb_a_r.withColumn('price_norm', (F.col('price_log') - price_log_mean) / (price_log_std + 1e-8))

#### Elapsed days (log)

In [22]:
%%time
product_recency_days_log_mean, product_recency_days_log_std = tuple(_df_sb_a_r.agg(F.mean('product_recency_days_log'), F.stddev('product_recency_days_log')).take(1)[0])
print(product_recency_days_log_mean, product_recency_days_log_std)

3.2383064627782043 0.8394709773058031
CPU times: user 6.41 ms, sys: 16.4 ms, total: 22.8 ms
Wall time: 1min 19s


In [23]:
#Z-norm
_df_sb_a_r = _df_sb_a_r.withColumn('product_recency_days_norm', (F.col('product_recency_days_log') - product_recency_days_log_mean) / (product_recency_days_log_std + 1e-8))

## Processing sessions sequences

#### Aggregate by session id (create sequence as type of array)

In [24]:
session_window = Window.partitionBy('user_session').orderBy('event_time_ts')

In [25]:
_df_sb_a_wprevt = _df_sb_a_r.withColumn('prev_event_time_ts', F.lag('event_time_ts').over(session_window)) \
                            .withColumn('delta_event_time',F.when(F.isnull(F.col('prev_event_time_ts')), 0) \
                                                            .otherwise(F.col('event_time_ts') - F.col('prev_event_time_ts')))

#### Delta time between session clicks (secs)

In [26]:
%%time
session_delta_time_mean, session_delta_time_std = tuple(_df_sb_a_wprevt.agg(
                            F.mean('delta_event_time').alias('delta_event_time_mean'),
                            F.stddev('delta_event_time').alias('delta_event_time_std')).take(1)[0])
print(session_delta_time_mean, session_delta_time_std)

#Z-norm
_df_sb_a_wprevt = _df_sb_a_wprevt.withColumn('delta_event_time_norm', (F.col('delta_event_time') - session_delta_time_mean) / session_delta_time_std)

230.3930558795769 10024.639844770056
CPU times: user 5.86 ms, sys: 35.2 ms, total: 41 ms
Wall time: 2min 52s


In [27]:
_df_sb_a_wprevt.printSchema()

root
 |-- category_id: long (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- event_time_dt: timestamp (nullable = true)
 |-- event_time_ts: long (nullable = true)
 |-- et_hour: integer (nullable = true)
 |-- et_dayofweek: integer (nullable = true)
 |-- et_dayofmonth: integer (nullable = true)
 |-- et_month: integer (nullable = true)
 |-- et_hour_sin: float (nullable = true)
 |-- et_hour_cos: float (nullable = true)
 |-- et_dayofweek_sin: float (nullable = true)
 |-- et_dayofweek_cos: float (nullable = true)
 |-- et_dayofmonth_sin: float (nullable = true)
 |-- et_dayofmonth_cos: float (nullable = true)
 |-- et_month_sin: float (nullable = true)
 |-- et_month_cos: float (nullable = true)
 

In [28]:
def get_non_repeated_items(values):
    #Returns unique items, keep the order of their first occurence
    result = []
    for v in values:
        if v not in result:
            result.append(v)
    return result

@udf(returnType=ArrayType(IntegerType()))
def get_non_repeated_items_integer_udf(values):
    result = get_non_repeated_items(values)
    result = list([int(x) for x in result])
    return result

#########################################

def get_non_repeated_additional_items(item_ids, additional_feature_values):
    #Returns unique items, keep the order of their first occurence
    ids = []
    result = []
    for i, v in zip(item_ids, additional_feature_values):
        if i not in ids:
            ids.append(i)
            result.append(v)
    return result

@udf(returnType=ArrayType(IntegerType()))
def get_non_repeated_additional_items_integer_udf(item_ids, additional_feature_values):
    result = get_non_repeated_additional_items(item_ids, additional_feature_values)
    result = list([int(x) for x in result])
    return result

@udf(returnType=ArrayType(FloatType()))
def get_non_repeated_additional_items_float_udf(item_ids, additional_feature_values):
    result =  get_non_repeated_additional_items(item_ids, additional_feature_values)
    result = list([float(x) for x in result])
    return result

In [29]:
df_sb_grouped_df = _df_sb_a_wprevt \
                .select('user_idx', 'user_session', 
                        F.first('event_time_ts').over(session_window).alias('session_start_ts'),
                        F.last('event_time_ts').over(session_window).alias('session_end_ts'),
                        F.collect_list('product_idx').over(session_window).alias('pid_seq_als'), \
                        F.collect_list('event_time_ts').over(session_window).alias('etime_seq_als'),\
                        F.collect_list('event_type_idx').over(session_window).alias('event_type_seq_als'), \
                        F.collect_list('category_sub_idx').over(session_window).alias('csid_seq_als'),\
                        F.collect_list('category_code_idx').over(session_window).alias('ccid_seq_als'),\
                        F.collect_list('brand_idx').over(session_window).alias('bid_seq_als'),\
                        F.collect_list('price_norm').over(session_window).alias('price_seq_als'),\
                        F.collect_list('delta_event_time_norm').over(session_window).alias('dtime_seq_als'),\
                        F.collect_list('product_recency_days_norm').over(session_window).alias('prod_recency_days_als'),\
                        F.collect_list('relative_price_to_avg_category_id').over(session_window).alias('relative_price_to_avg_category_als'),
                        F.collect_list('et_hour_sin').over(session_window).alias('et_hour_sin_seq_als'),\
                        F.collect_list('et_hour_cos').over(session_window).alias('et_hour_cos_seq_als'),\
                        F.collect_list('et_month_sin').over(session_window).alias('et_month_sin_seq_als'),\
                        F.collect_list('et_month_cos').over(session_window).alias('et_month_cos_seq_als'),\
                        F.collect_list('et_dayofweek_sin').over(session_window).alias('et_dayofweek_sin_seq_als'),\
                        F.collect_list('et_dayofweek_cos').over(session_window).alias('et_dayofweek_cos_seq_als'),\
                        F.collect_list('et_dayofmonth_sin').over(session_window).alias('et_dayofmonth_sin_seq_als'),\
                        F.collect_list('et_dayofmonth_cos').over(session_window).alias('et_dayofmonth_cos_seq_als'),\
                        
                       )\
                .groupBy('user_idx', 'user_session').agg(
                    F.min('session_start_ts').alias('session_start_ts'),
                    F.max('session_end_ts').alias('session_end_ts'),
                    F.max('pid_seq_als').alias('sess_pid_seq'),
                    F.max('etime_seq_als').alias('sess_etime_seq'),
                    F.max('event_type_seq_als').alias('sess_etype_seq'),    
                    F.max('csid_seq_als').alias('sess_csid_seq'),
                    F.max('ccid_seq_als').alias('sess_ccid_seq'),
                    F.max('bid_seq_als').alias('sess_bid_seq'),
                    F.max('price_seq_als').alias('sess_price_seq'),                    
                    F.max('dtime_seq_als').alias('sess_dtime_seq'),
                    F.max('prod_recency_days_als').alias('sess_product_recency_seq'),
                    F.max('relative_price_to_avg_category_als').alias('sess_relative_price_to_avg_category_seq'),
                    F.max('et_hour_sin_seq_als').alias('sess_et_hour_sin_seq'),
                    F.max('et_hour_cos_seq_als').alias('sess_et_hour_cos_seq'),
                    F.max('et_month_sin_seq_als').alias('sess_et_month_sin_seq'),
                    F.max('et_month_cos_seq_als').alias('sess_et_month_cos_seq'),
                    F.max('et_dayofweek_sin_seq_als').alias('sess_et_dayofweek_sin_seq'),
                    F.max('et_dayofweek_cos_seq_als').alias('sess_et_dayofweek_cos_seq'),
                    F.max('et_dayofmonth_sin_seq_als').alias('sess_et_dayofmonth_sin_seq'),
                    F.max('et_dayofmonth_cos_seq_als').alias('sess_et_dayofmonth_cos_seq'),               
                        )

In [30]:
df_sb_grouped_nr_df = df_sb_grouped_df.select('user_idx', 'user_session', 'session_start_ts', 'session_end_ts',
                        get_non_repeated_items_integer_udf('sess_pid_seq').alias('sess_pid_seq'),
                        get_non_repeated_additional_items_integer_udf(F.col("sess_pid_seq"), F.col('sess_etime_seq')).alias('sess_etime_seq'),
                        get_non_repeated_additional_items_integer_udf(F.col("sess_pid_seq"), F.col('sess_etype_seq')).alias('sess_etype_seq'),                        
                        get_non_repeated_additional_items_integer_udf(F.col("sess_pid_seq"), F.col('sess_csid_seq')).alias('sess_csid_seq'),
                        get_non_repeated_additional_items_integer_udf(F.col("sess_pid_seq"), F.col('sess_ccid_seq')).alias('sess_ccid_seq'),
                        get_non_repeated_additional_items_integer_udf(F.col("sess_pid_seq"), F.col('sess_bid_seq')).alias('sess_bid_seq'),
                        get_non_repeated_additional_items_float_udf(F.col("sess_pid_seq"), F.col('sess_price_seq')).alias('sess_price_seq'),                                              
                        get_non_repeated_additional_items_float_udf(F.col("sess_pid_seq"), F.col('sess_dtime_seq')).alias('sess_dtime_seq'),
                        get_non_repeated_additional_items_float_udf(F.col("sess_pid_seq"), F.col('sess_product_recency_seq')).alias('sess_product_recency_seq'),
                        get_non_repeated_additional_items_float_udf(F.col("sess_pid_seq"), F.col('sess_relative_price_to_avg_category_seq')).alias('sess_relative_price_to_avg_category_seq'),
                        get_non_repeated_additional_items_float_udf(F.col("sess_pid_seq"), F.col('sess_et_hour_sin_seq')).alias('sess_et_hour_sin_seq'),
                        get_non_repeated_additional_items_float_udf(F.col("sess_pid_seq"), F.col('sess_et_hour_cos_seq')).alias('sess_et_hour_cos_seq'),
                        get_non_repeated_additional_items_float_udf(F.col("sess_pid_seq"), F.col('sess_et_month_sin_seq')).alias('sess_et_month_sin_seq'),
                        get_non_repeated_additional_items_float_udf(F.col("sess_pid_seq"), F.col('sess_et_month_cos_seq')).alias('sess_et_month_cos_seq'),
                        get_non_repeated_additional_items_float_udf(F.col("sess_pid_seq"), F.col('sess_et_dayofweek_sin_seq')).alias('sess_et_dayofweek_sin_seq'),
                        get_non_repeated_additional_items_float_udf(F.col("sess_pid_seq"), F.col('sess_et_dayofweek_cos_seq')).alias('sess_et_dayofweek_cos_seq'),
                        get_non_repeated_additional_items_float_udf(F.col("sess_pid_seq"), F.col('sess_et_dayofmonth_sin_seq')).alias('sess_et_dayofmonth_sin_seq'),
                        get_non_repeated_additional_items_float_udf(F.col("sess_pid_seq"), F.col('sess_et_dayofmonth_cos_seq')).alias('sess_et_dayofmonth_cos_seq'),
                       ) \
            .where(F.size('sess_pid_seq') >= 2) \
            .withColumn('sess_seq_len', F.size('sess_pid_seq'))        

In [31]:
df_sb_grouped_nr_df.printSchema()

root
 |-- user_idx: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- session_start_ts: long (nullable = true)
 |-- session_end_ts: long (nullable = true)
 |-- sess_pid_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_etime_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_etype_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_csid_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_ccid_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_bid_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_price_seq: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- sess_dtime_seq: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- sess_product_recency_seq: array (nullable = true)
 |    |-- element: floa

In [40]:
#df_sb_grouped_nr_df.show()

+--------+--------------------+----------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------+------------------------+---------------------------------------+--------------------+--------------------+---------------------+---------------------+-------------------------+-------------------------+--------------------------+--------------------------+------------+
|user_idx|        user_session|session_start_ts|session_end_ts|        sess_pid_seq|      sess_etime_seq|sess_etype_seq|       sess_csid_seq|       sess_ccid_seq|        sess_bid_seq|      sess_price_seq|sess_dtime_seq|sess_product_recency_seq|sess_relative_price_to_avg_category_seq|sess_et_hour_sin_seq|sess_et_hour_cos_seq|sess_et_month_sin_seq|sess_et_month_cos_seq|sess_et_dayofweek_sin_seq|sess_et_dayofweek_cos_seq|sess_et_dayofmonth_sin_seq|sess_et_dayofmonth_cos_seq|sess_seq_len|
+--------+----------

In [32]:
percentiles = np.arange(0., 1.1, 0.1).tolist()+[0.95,0.99,0.999]

In [96]:
session_sizes_pdf = pd.DataFrame(zip(percentiles, 
                                          df_sb_grouped_nr_df.approxQuantile('sess_seq_len', probabilities=percentiles, relativeError=0.00001)),
                                         columns=['percentile', 'session_sizes']).sort_values('percentile')
session_sizes_pdf

,percentile,session_sizes
0,0.000,2.0
1,0.100,2.0
2,0.200,2.0
3,0.300,2.0
4,0.400,2.0
5,0.500,3.0
6,0.600,3.0
7,0.700,4.0
8,0.800,4.0
9,0.900,6.0


## Processing users sequences

In [33]:
# by manual
SESSIONS_MAX_LENGTH = 20

In [34]:
user_window = Window.partitionBy('user_idx').orderBy('event_time_ts')

#### Delta time between session clicks (secs)

In [35]:
_df_user_wprevt = _df_sb_a_r.withColumn('user_prev_event_time_ts', F.lag('event_time_ts').over(user_window)) \
                            .withColumn('user_delta_event_time',F.when(F.isnull(F.col('user_prev_event_time_ts')), 0) \
                                                            .otherwise(F.col('event_time_ts') - F.col('user_prev_event_time_ts'))) \
                            .withColumn('user_delta_event_time_log', F.log1p('user_delta_event_time'))

In [36]:
%%time
user_delta_time_mean, user_delta_time_std = tuple(_df_user_wprevt.agg(
                            F.mean('user_delta_event_time_log').alias('user_delta_event_time_mean'),
                            F.stddev('user_delta_event_time_log').alias('user_delta_event_time_std')).take(1)[0])
print(user_delta_time_mean, user_delta_time_std)

#Z-norm
_df_user_wprevt = _df_user_wprevt.withColumn('user_delta_event_time_log_norm', (F.col('user_delta_event_time_log') - user_delta_time_mean) / user_delta_time_std)

4.465940900094787 3.026454528930501
CPU times: user 11.6 ms, sys: 15.5 ms, total: 27.1 ms
Wall time: 1min 36s


In [37]:
df_user_seq_grouped_df = _df_user_wprevt \
                .select('user_idx',  
                        F.collect_list('product_idx').over(user_window).alias('pid_seq_als'), 
                        F.collect_list('event_time_ts').over(user_window).alias('etime_seq_als'),
                        F.collect_list('event_type_idx').over(user_window).alias('event_type_seq_als'), 
                        F.collect_list('category_sub_idx').over(user_window).alias('csid_seq_als'),
                        F.collect_list('category_code_idx').over(user_window).alias('ccid_seq_als'),
                        F.collect_list('brand_idx').over(user_window).alias('bid_seq_als'),
                        F.collect_list('price_norm').over(user_window).alias('price_seq_als'),
                        F.collect_list('user_delta_event_time_log_norm').over(user_window).alias('dtime_seq_als'),
                        F.collect_list('product_recency_days_norm').over(user_window).alias('prod_recency_days_als'),
                        F.collect_list('relative_price_to_avg_category_id').over(user_window).alias('relative_price_to_avg_category_als'),
                        F.collect_list('et_hour_sin').over(user_window).alias('et_hour_sin_seq_als'),
                        F.collect_list('et_hour_cos').over(user_window).alias('et_hour_cos_seq_als'),
                        F.collect_list('et_month_sin').over(user_window).alias('et_month_sin_seq_als'),
                        F.collect_list('et_month_cos').over(user_window).alias('et_month_cos_seq_als'),
                        F.collect_list('et_dayofweek_sin').over(user_window).alias('et_dayofweek_sin_seq_als'),
                        F.collect_list('et_dayofweek_cos').over(user_window).alias('et_dayofweek_cos_seq_als'),
                        F.collect_list('et_dayofmonth_sin').over(user_window).alias('et_dayofmonth_sin_seq_als'),
                        F.collect_list('et_dayofmonth_cos').over(user_window).alias('et_dayofmonth_cos_seq_als'),                     
                       )\
                .groupBy('user_idx').agg(
                    F.max('pid_seq_als').alias('user_pid_seq'),
                    F.max('etime_seq_als').alias('user_etime_seq'),
                    F.max('event_type_seq_als').alias('user_etype_seq'),    
                    F.max('csid_seq_als').alias('user_csid_seq'),
                    F.max('ccid_seq_als').alias('user_ccid_seq'),
                    F.max('bid_seq_als').alias('user_bid_seq'),
                    F.max('price_seq_als').alias('user_price_seq'),                    
                    F.max('dtime_seq_als').alias('user_dtime_seq'),
                    F.max('prod_recency_days_als').alias('user_product_recency_seq'),
                    F.max('relative_price_to_avg_category_als').alias('user_relative_price_to_avg_category_seq'),
                    F.max('et_hour_sin_seq_als').alias('user_et_hour_sin_seq'),
                    F.max('et_hour_cos_seq_als').alias('user_et_hour_cos_seq'),
                    F.max('et_month_sin_seq_als').alias('user_et_month_sin_seq'),
                    F.max('et_month_cos_seq_als').alias('user_et_month_cos_seq'),
                    F.max('et_dayofweek_sin_seq_als').alias('user_et_dayofweek_sin_seq'),
                    F.max('et_dayofweek_cos_seq_als').alias('user_et_dayofweek_cos_seq'),
                    F.max('et_dayofmonth_sin_seq_als').alias('user_et_dayofmonth_sin_seq'),
                    F.max('et_dayofmonth_cos_seq_als').alias('user_et_dayofmonth_cos_seq'),               
                        )

In [38]:
df_user_seq_grouped_df.printSchema()

root
 |-- user_idx: integer (nullable = true)
 |-- user_pid_seq: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- user_etime_seq: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- user_etype_seq: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- user_csid_seq: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- user_ccid_seq: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- user_bid_seq: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- user_price_seq: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- user_dtime_seq: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- user_product_recency_seq: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- user_relative_price_to_avg_category_seq: array (nullable = true)
 |    |-- element: double (containsNull = tru

### Joining users and sessions sequences

In [39]:
users_and_session_seq_joined_df = df_sb_grouped_nr_df.join(df_user_seq_grouped_df, on='user_idx', how='inner')

In [40]:
users_and_session_seq_joined_df.printSchema()

root
 |-- user_idx: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- session_start_ts: long (nullable = true)
 |-- session_end_ts: long (nullable = true)
 |-- sess_pid_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_etime_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_etype_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_csid_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_ccid_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_bid_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_price_seq: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- sess_dtime_seq: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- sess_product_recency_seq: array (nullable = true)
 |    |-- element: floa

In [41]:
def user_seq_feature_before_session(session_start, user_ts_seq, user_feature_seq, limit):
    new_seq = list(map(lambda y: y[1], filter(lambda x: x[0] < session_start, zip(user_ts_seq, user_feature_seq))))[-limit:]
    return new_seq

@udf(returnType=ArrayType(IntegerType()))
def user_seq_before_session_integer_udf(session_start, user_ts_seq, user_feature_seq, limit):
    result = user_seq_feature_before_session(session_start, user_ts_seq, user_feature_seq, limit)
    return list([int(x) for x in result])

@udf(returnType=ArrayType(FloatType()))
def user_seq_before_session_float_udf(session_start, user_ts_seq, user_feature_seq, limit):
    result = user_seq_feature_before_session(session_start, user_ts_seq, user_feature_seq, limit)
    return list([float(x) for x in result])

In [42]:
MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION = 20

In [43]:
#Assume that users with unseen previous interactions within the dataset (6 months) interacted 7 months before
ELAPSED_DAYS_BEF_LAST_SESSION_DEFAULT = 7*30 

@udf(returnType=FloatType())
def elapsed_days_since_last_interaction_bef_session(session_start, previous_interactions_ts_list):
    if len(previous_interactions_ts_list) == 0:
        return ELAPSED_DAYS_BEF_LAST_SESSION_DEFAULT
    
    elapsed_days = max(session_start - previous_interactions_ts_list[-1], 0) / (60*60*24)
    return elapsed_days

In [44]:
session_seq_and_prev_users_seq_df = users_and_session_seq_joined_df \
            .withColumn('user_pid_seq_bef_sess', user_seq_before_session_integer_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_pid_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_etime_seq_bef_sess', user_seq_before_session_integer_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_etime_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_etype_seq_bef_sess', user_seq_before_session_integer_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_etype_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_csid_seq_bef_sess', user_seq_before_session_integer_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_csid_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_ccid_seq_bef_sess', user_seq_before_session_integer_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_ccid_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_bid_seq_bef_sess', user_seq_before_session_integer_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_bid_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_price_seq_bef_sess', user_seq_before_session_float_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_price_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_dtime_seq_bef_sess', user_seq_before_session_float_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_dtime_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_product_recency_seq_bef_sess', user_seq_before_session_float_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_product_recency_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_relative_price_to_avg_category_seq_bef_sess', user_seq_before_session_float_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_relative_price_to_avg_category_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_et_hour_sin_seq_bef_sess', user_seq_before_session_float_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_et_hour_sin_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_et_hour_cos_seq_bef_sess', user_seq_before_session_float_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_et_hour_cos_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_et_month_sin_seq_bef_sess', user_seq_before_session_float_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_et_month_sin_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_et_month_cos_seq_bef_sess', user_seq_before_session_float_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_et_month_cos_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_et_dayofweek_sin_seq_bef_sess', user_seq_before_session_float_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_et_dayofweek_sin_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_et_dayofweek_cos_seq_bef_sess', user_seq_before_session_float_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_et_dayofweek_cos_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_et_dayofmonth_sin_seq_bef_sess', user_seq_before_session_float_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_et_dayofmonth_sin_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_et_dayofmonth_cos_seq_bef_sess', user_seq_before_session_float_udf(F.col('session_start_ts'), F.col('user_etime_seq'),  F.col('user_et_dayofmonth_cos_seq'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION))) \
            .withColumn('user_seq_length_bef_sess', F.size('user_pid_seq_bef_sess')) \
            .withColumn('user_elapsed_days_bef_sess', elapsed_days_since_last_interaction_bef_session(F.col('session_start_ts'), F.col('sess_etime_seq'))) \
            .withColumn('user_elapsed_days_log_bef_sess', F.log1p('user_elapsed_days_bef_sess'))

In [45]:
%%time
user_elapsed_days_log_bef_sess_mean, user_elapsed_days_log_bef_sess_std = tuple(session_seq_and_prev_users_seq_df.agg(
                                                                                F.mean('user_elapsed_days_log_bef_sess'),
                                                                                F.stddev('user_elapsed_days_log_bef_sess')).take(1)[0])
print(user_elapsed_days_log_bef_sess_mean, user_elapsed_days_log_bef_sess_std)

#Z-norm
session_seq_and_prev_users_seq_df = session_seq_and_prev_users_seq_df.withColumn('user_elapsed_days_log_bef_sess_norm', 
                                                                                 (F.col('user_elapsed_days_log_bef_sess') - user_elapsed_days_log_bef_sess_mean) / user_elapsed_days_log_bef_sess_std)

0.0 0.0
CPU times: user 50.5 ms, sys: 69.6 ms, total: 120 ms
Wall time: 4min 32s


In [46]:
session_seq_and_prev_users_seq_df.printSchema()

root
 |-- user_idx: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- session_start_ts: long (nullable = true)
 |-- session_end_ts: long (nullable = true)
 |-- sess_pid_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_etime_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_etype_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_csid_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_ccid_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_bid_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_price_seq: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- sess_dtime_seq: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- sess_product_recency_seq: array (nullable = true)
 |    |-- element: floa

# Exporting data

#### Option A: Pad zeros to make all sequences same length
NOTE: it would be not memory-efficient. But currently it seems no way to utilize Petastorm (parquet -> dataloader) without this option.

In [47]:
from pyspark.sql.types import ArrayType, IntegerType

def pad_array(values, expected_length, trunc_start, dtype=int):
    #if dtype is float:
    #    value_to_pad = [0.]
    #else:
    #    value_to_pad = [0]
    #value_to_pad = [dtype(0)]
        
    res = list([type(x) for x in values]) + ([dtype(0)] * (expected_length-len(values)))
    
    # Pick first N
    if trunc_start:
        res = res[:expected_length]
    #Pick last N
    else:
        res = res[-expected_length:]
        
    return res


@udf(returnType=ArrayType(IntegerType()))
def pad_array_int(values, expected_length, trunc_start):
    return pad_array(values, expected_length, trunc_start)

@udf(returnType=ArrayType(LongType()))
def pad_array_long(values, expected_length, trunc_start):
    return pad_array(values, expected_length, trunc_start)

@udf(returnType=ArrayType(FloatType()))
def pad_array_float(values, expected_length, trunc_start):
    return pad_array(values, expected_length, trunc_start, dtype=float)

In [48]:
sessions_users_seqs_to_export_df = \
        session_seq_and_prev_users_seq_df.select('user_idx', 'user_session', 'sess_seq_len', 
                                                 'session_start_ts',  
                                                 F.date_trunc('day', F.to_timestamp(F.col('session_start_ts'))).cast('string').substr(0,10).alias('session_start_date'), 
                                                 'user_seq_length_bef_sess', 'user_elapsed_days_bef_sess', 'user_elapsed_days_log_bef_sess_norm',
                        # Session sequences (first N interactions)
                        pad_array_long(F.col('sess_pid_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_pid_seq'),
                        pad_array_long(F.col('sess_etime_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_etime_seq'),
                        pad_array_int(F.col('sess_etype_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_etype_seq'),
                        pad_array_int(F.col('sess_csid_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_csid_seq'),
                        pad_array_int(F.col('sess_ccid_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_ccid_seq'),
                        pad_array_int(F.col('sess_bid_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_bid_seq'),
                        pad_array_float(F.col('sess_price_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_price_seq'),
                        pad_array_float(F.col('sess_dtime_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_dtime_seq'),
                        pad_array_float(F.col('sess_product_recency_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_product_recency_seq'),
                        pad_array_float(F.col('sess_relative_price_to_avg_category_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_relative_price_to_avg_category_seq'),
                        pad_array_float(F.col('sess_et_hour_sin_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_et_hour_sin_seq'), 
                        pad_array_float(F.col('sess_et_hour_cos_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_et_hour_cos_seq'),
                        pad_array_float(F.col('sess_et_month_sin_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_et_month_sin_seq'),
                        pad_array_float(F.col('sess_et_month_cos_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_et_month_cos_seq'),
                        pad_array_float(F.col('sess_et_dayofweek_sin_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_et_dayofweek_sin_seq'),
                        pad_array_float(F.col('sess_et_dayofweek_cos_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_et_dayofweek_cos_seq'),
                        pad_array_float(F.col('sess_et_dayofmonth_sin_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_et_dayofmonth_sin_seq'),
                        pad_array_float(F.col('sess_et_dayofmonth_cos_seq'), F.lit(SESSIONS_MAX_LENGTH), F.lit(True)).alias('sess_et_dayofmonth_cos_seq'),
                         # Users sequences before session (last M interactions) 
                         pad_array_long(F.col('user_pid_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_pid_seq_bef_sess'),
                         pad_array_long(F.col('user_etime_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_etime_seq_bef_sess'),
                         pad_array_int(F.col('user_etype_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_etype_seq_bef_sess'),
                         pad_array_int(F.col('user_csid_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_csid_seq_bef_sess'),
                         pad_array_int(F.col('user_ccid_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_ccid_seq_bef_sess'),
                         pad_array_int(F.col('user_bid_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_bid_seq_bef_sess'),
                         pad_array_float(F.col('user_price_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_price_seq_bef_sess'),
                         pad_array_float(F.col('user_dtime_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_dtime_seq_bef_sess'),
                         pad_array_float(F.col('user_product_recency_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_product_recency_seq_bef_sess'),
                         pad_array_float(F.col('user_relative_price_to_avg_category_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_relative_price_to_avg_category_seq_bef_sess'),
                         pad_array_float(F.col('user_et_hour_sin_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_et_hour_sin_seq_bef_sess'),
                         pad_array_float(F.col('user_et_hour_cos_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_et_hour_cos_seq_bef_sess'),
                         pad_array_float(F.col('user_et_month_sin_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_et_month_sin_seq_bef_sess'),
                         pad_array_float(F.col('user_et_month_cos_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_et_month_cos_seq_bef_sess'),
                         pad_array_float(F.col('user_et_dayofweek_sin_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_et_dayofweek_sin_seq_bef_sess'),
                         pad_array_float(F.col('user_et_dayofweek_cos_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_et_dayofweek_cos_seq_bef_sess'),
                         pad_array_float(F.col('user_et_dayofmonth_sin_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_et_dayofmonth_sin_seq_bef_sess'),
                         pad_array_float(F.col('user_et_dayofmonth_cos_seq_bef_sess'), F.lit(MAX_LENGTH_USER_SEQUENCE_BEFORE_SESSION), F.lit(False)).alias('user_et_dayofmonth_cos_seq_bef_sess'),
                    )

In [49]:
sessions_users_seqs_to_export_df.printSchema()

root
 |-- user_idx: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- sess_seq_len: integer (nullable = false)
 |-- session_start_ts: long (nullable = true)
 |-- session_start_date: string (nullable = true)
 |-- user_seq_length_bef_sess: integer (nullable = false)
 |-- user_elapsed_days_bef_sess: float (nullable = true)
 |-- user_elapsed_days_log_bef_sess_norm: double (nullable = true)
 |-- sess_pid_seq: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- sess_etime_seq: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- sess_etype_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_csid_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_ccid_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_bid_seq: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- sess_price_seq: array (nullab

In [245]:
#sessions_users_seqs_to_export_df.show()

+--------+--------------------+------------+----------------+------------------+------------------------+------------------------------+-----------------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+---------------------------------------+--------------------+--------------------+---------------------+---------------------+-------------------------+-------------------------+--------------------------+--------------------------+---------------------+-----------------------+-----------------------+----------------------+----------------------+---------------------+-----------------------+-----------------------+---------------------------------+------------------------------------------------+-----------------------------+-----------------------------+------------------------------+------------------------------+------------

#### Export dataset to parquet, partioned by the session date

In [50]:
%%time
OUTPUT_PATH = '/home/jovyan/'
sessions_users_seqs_to_export_df.repartition(F.col('session_start_date')) \
                .write.partitionBy('session_start_date') \
                .parquet(os.path.join(OUTPUT_PATH, 'ecommerce_preproc.parquet'))

CPU times: user 294 ms, sys: 298 ms, total: 592 ms
Wall time: 33min 11s


In [48]:
1

1

In [50]:
temp_df = spark.read.parquet('ecommerce_preproc.parquet')

In [59]:
temp_df.take(1)

[Row(user_idx=476651191, user_session='5308244f-2d97-475d-9368-e98cb2fcaf6a', sess_seq_len=3, session_start_ts=1573383799, user_seq_length_bef_sess=26, user_elapsed_days_log_bef_sess=-0.0012507818736779838, user_elapsed_days_log_bef_sess_norm=0.15898386347431748, sess_pid_seq=[None, None, None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], sess_etime_seq=[1573383799, 1573383872, 1573383907, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], sess_etype_seq=[None, None, None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], sess_csid_seq=[None, None, None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], sess_ccid_seq=[None, None, None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], sess_bid_seq=[None, None, None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], sess_price_seq=[-1.6059000492095947, -1.8523046970367432, -1.3824394941329956, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], sess_dtime_seq=[-0.022982677444815636, -0.0